# Data Acquisition from Yahoo Finance

In [3]:
import yfinance as yf

symbol = 'MSFT'
data = yf.download(symbol, start='2020-01-01', end='2025-01-01')
data.dropna(inplace=True)
print(data.tail())

[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open    Volume
Ticker            MSFT        MSFT        MSFT        MSFT      MSFT
Date                                                                
2024-12-24  438.450836  438.720315  433.321138  433.780209   7164500
2024-12-26  437.233276  440.057630  435.756258  438.201337   8194200
2024-12-27  429.668457  434.349074  425.496829  433.730320  18117700
2024-12-30  423.979858  426.694417  421.055729  425.207408  13158700
2024-12-31  420.656525  425.876070  419.818209  425.247326  13246500


# Preparing Data for Backtesting

In [5]:
import pandas as pd

data['Returns'] = data['Close'].pct_change()
data.dropna(inplace=True)
print(data.tail())

Price            Close        High         Low        Open    Volume   Returns
Ticker            MSFT        MSFT        MSFT        MSFT      MSFT          
Date                                                                          
2024-12-24  438.450836  438.720315  433.321138  433.780209   7164500  0.009374
2024-12-26  437.233276  440.057630  435.756258  438.201337   8194200 -0.002777
2024-12-27  429.668457  434.349074  425.496829  433.730320  18117700 -0.017302
2024-12-30  423.979858  426.694417  421.055729  425.207408  13158700 -0.013240
2024-12-31  420.656525  425.876070  419.818209  425.247326  13246500 -0.007838


# Implementing Backtesting using Python (Backtrader)

In [7]:
import backtrader as bt
import pandas as pd

# Flatten MultiIndex columns if needed
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

class GoldenCrossStrategy(bt.Strategy):
    def __init__(self):
        self.sma_short = bt.indicators.SMA(self.datas[0].close, period=50)
        self.sma_long = bt.indicators.SMA(self.datas[0].close, period=200)
        self.buy_price = None
        self.order = None
        self.trade_count = 0

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buy_price = order.executed.price
                print(f"BUY EXECUTED: {order.executed.size} shares at {self.buy_price:.2f}")
            elif order.issell():
                print(f"SELL EXECUTED: {order.executed.size} shares at {order.executed.price:.2f}")
                self.buy_price = None
            self.order = None
            self.trade_count += 1
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print("Order Canceled or Rejected")
            self.order = None

    def next(self):
        if len(self) < 201:  # Wait until indicators are ready
            return

        current_price = self.datas[0].close[0]

        if self.order:
            return

        # Entry condition: SMA50 above SMA200
        if not self.position and self.sma_short[0] > self.sma_long[0]:
            size = int(self.broker.getcash() // current_price)
            if size > 0:
                self.order = self.buy(size=size)

        # Exit conditions: Take profit, stop loss, or SMA reversal
        elif self.position and self.buy_price:
            take_profit_price = self.buy_price * 1.20
            stop_loss_price = self.buy_price * 0.90

            if current_price >= take_profit_price:
                self.order = self.sell()
            elif current_price <= stop_loss_price:
                self.order = self.sell()
            elif self.sma_short[0] < self.sma_long[0]:
                self.order = self.sell()

# Backtrader setup
cerebro = bt.Cerebro()
cerebro.addstrategy(GoldenCrossStrategy)

data_feed = bt.feeds.PandasData(dataname=data)
cerebro.adddata(data_feed)

cerebro.broker.setcash(10000)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print(f"Total Trades Executed: {results[0].trade_count}")


Starting Portfolio Value: 10000.00
BUY EXECUTED: 48 shares at 207.64
SELL EXECUTED: -1 shares at 248.96
Final Portfolio Value: 20053.33
Total Trades Executed: 2


# Analyzing Backtesting Results

In [9]:
print(f"Total Return: {(cerebro.broker.getvalue() - 10000) / 10000 * 100:.2f}%")

Total Return: 100.53%
